In [ ]:
import os, sys, math, scipy
import pandas as pd
import numpy as np
from scipy.spatial import distance

path = os.getcwd()
optpath = path + '/optimization_data'


In [ ]:
if 'Young_Research' in path:
    sys.path.append("C:\\Users\\Young_Research\\Documents\\Rutgers\\Research\\Scripts\\analysis\\")
    ncppath = "C:\\Users\\Young_Research\\Documents\\Rutgers\\Research\\nuc_database\\"
else:
    sys.path.append("/home/rty10/Documents/scripts/analysis/")
    ncppath = '/home/rty10/Documents/nuc_database/'
import dna_analysis

# Make sure to import ncms_data_compile to collect westcott angle information for minichrom circles
#import circle_data_compile as opt_compile
import ncms_data_compile as opt_compile

ncpcluster = pd.read_csv(ncppath+"ncps_twclusters_ry.csv", index_col=False)
NCPID = {ncpcluster.at[i, 'pdb_id']:ncpcluster.at[i, 'twist_cluster'] for i in range(len(ncpcluster))}
del ncpcluster

# Compile: all opt data to single .csv

In [ ]:
loop_startpts = {'n336':141, 'n341':141, 'n359':141}
loop_midpts   = {'n336':141+int((336-141)/2),'n341':141+int((341-141)/2), 'n359':141+int((359-141)/2)}
loop_endpts   = {'n336':336, 'n341':341, 'n359':359}

In [ ]:
CSV_NAME = "2021Dec_ncms_delLK_n336"

In [ ]:
circles = [ filename.split('.')[0] for filename in os.listdir(optpath) if filename.endswith('.log') ]

circles=sorted(circles)

print(circles[0:5])


In [ ]:
# Error checkings
for filename in sorted(os.listdir(optpath)):
    if filename.endswith('.log'):
        
        infile = open(optpath+'/'+filename, "r")
        data = infile.readlines()
        data = [i.rstrip('\n') for i in data]
        infile.close()        
        for line in data:
            if 'return code: ' in line:
                if line.split(': ')[1] != 'EPSG':
                    print(filename, line.split(':')[1])

        del data

In [ ]:
df = pd.DataFrame()
Nnuc = 141
for i in range(len(circles)):
    name = circles[i]    
    df.at[name, 'nuc']        = name.split('_')[0]
    df.at[name, 'lko']        = int( name.split('_')[2][2:] )
    L_ncms = int( name.split('_')[1].lstrip('n') )
    L_loop = L_ncms - int(Nnuc) #breathdict[ name.split('_')[3] ][0]
    df.at[name, 'ncms_size']  = L_ncms
    df.at[name, 'looplen']    = L_loop
    df.at[name, 'nuc_turns']   = (L_ncms - L_loop)/80
    df.at[name, 'twist_cluster'] = NCPID[name.split('_')[0]]
    
    df = opt_compile.log_file_data(optpath+'/'+name+'.log', name, df)
    df = opt_compile.topology_file_data(optpath+'/topo_'+name+'.txt', name, df)    
    df = opt_compile.refframe_file_data(optpath+'/'+name+'.dat', name, df,
                                        L_ncms, L_loop, 
                                        loop_startpts[name.split('_')[1]], 
                                        loop_midpts[name.split('_')[1]],
                                        loop_endpts[name.split('_')[1]])
    del L_ncms, L_loop, name

df['sh_density'] = (df['lk']-(df['ncms_size']/10.5))/(df['ncms_size']/10.5)

df[['lko','ncms_size','looplen','twist_cluster']] = df[['lko','ncms_size','looplen','twist_cluster']].astype(int)
df = df.sort_values(by=['nuc','lko'])
df = df.reset_index(drop=True)
del Nnuc
df.to_csv(CSV_NAME)
df.head(10)

In [ ]:
#CHECK if datafile was saved to .csv file

del df
df = pd.read_csv(CSV_NAME, index_col=0)
df.head(10)


In [ ]:
del df, circles

In [ ]:
df = pd.read_csv(CSV_NAME, index_col=0)
lst = df.nuc.unique()

df['exp_eopt']         = np.exp(-1*df['eopt'])
df['exp_eopt_elastic'] = np.exp(-1*df['eopt_elastic'])
df['partition_eopt']         = np.nan
df['partition_eopt_elastic'] = np.nan

for i in range(len(lst)):
    df1 = df.copy()
    df1 = df1[df1.nuc==lst[i]]
    INDEX = df1.index.values.tolist()
    A = df1['exp_eopt'].sum()
    B = df1['exp_eopt_elastic'].sum()
    for j in INDEX:
        df.at[j, 'partition_eopt']         = df1.at[j, 'exp_eopt']/A
        df.at[j, 'partition_eopt_elastic'] = df1.at[j, 'exp_eopt_elastic']/B
    del df1, INDEX, A, B

df.to_csv(CSV_NAME)
del lst, df

In [ ]:
df = pd.read_csv(CSV_NAME, index_col=0)
df

### Error Checking (29 Dec 2021)

In [ ]:
#CHECK if datafile was saved to .csv file

df = pd.read_csv(CSV_NAME, index_col=0)
infile = open("ncpid_list_2021Apr.txt", "r")
indata = infile.readlines()
infile.close()
indata = [i.strip('\n') for i in indata]
NCPS   = sorted(df.nuc.unique().tolist())

[print(i) for i in indata if i not in NCPS]

del indata, NCPS, df
